In [2]:
import numpy as np
import random
from tensorflow import keras
from keras import layers
from keras.utils import to_categorical
from keras.preprocessing import image
from PIL import Image
import os

# generowanie sztucznego zbioru uczącego - generowanie obrazków na podstawie źródła
root_folder = '.\\F1 Teams\\'
list_of_teams = os.listdir(root_folder)
class_to_name = {}
for i, team in enumerate(list_of_teams):
    class_to_name[i] = team


def create_dataset(image_name, folder_name, number_of_samples=250):
    for i in range(number_of_samples):
        try:
            with Image.open(f".\\{folder_name}\\{image_name}") as img:
                angle = random.uniform(0, 360)
                rotated_img = img.rotate(angle, expand=True)
                augmented_path = f".\\{folder_name}\\{folder_name.split("\\")[-1]}_{i}.jpg"
                final_image = rotated_img.resize((250, 250))
                final_image.save(augmented_path)

        except Exception as e:
            print(f"Error processing {image_name}: {e}")


for root, dirs, files in os.walk(root_folder):
    for file in files:
        if file == "logo.jpg":
            create_dataset(file, f"{root}")


{0: 'Alpine', 1: 'Aston Martin', 2: 'Ferrari', 3: 'Haas', 4: 'Kick Sauber', 5: 'McLaren', 6: 'Mercedes', 7: 'Racing Bulls', 8: 'Red Bull Racing', 9: 'Williams'}


In [5]:
number_of_classes = 10
image_size = (250, 250)
batch_size = 32

# podział danych na zbiory uczące i zbiory testowe
train_data = keras.utils.image_dataset_from_directory(
    "F1 teams",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

validation_data = keras.utils.image_dataset_from_directory(
    "F1 teams",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

# konwersja etykiet do danych kategorycznych
train_data = train_data.map(lambda x, y: (x/255.0, to_categorical(y, num_classes=number_of_classes)))
validation_data = validation_data.map(lambda x, y: (x/255.0, to_categorical(y, num_classes=number_of_classes)))


Found 2510 files belonging to 10 classes.
Using 2008 files for training.
Found 2510 files belonging to 10 classes.
Using 502 files for validation.


In [16]:
# definicja modelu
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(number_of_classes, activation='softmax')  # Output layer
])


model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 248, 248, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 59, 59, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │    27,558,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,653,962 (105.49 MB)

 Trainable params: 27,653,962 (105.49 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# uczenie modelu
model.compile(loss=keras.losses.CategoricalCrossentropy, optimizer="adam", metrics=["accuracy"])
model.fit(train_data, batch_size=32, epochs=15)

Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5041 - loss: 1.5753
Epoch 2/15
19/63 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.9832 - loss: 0.0643

KeyboardInterrupt: 

In [10]:
# ewaluacja modelu
evaluation_results = model.evaluate(validation_data)
print(f"Validation loss: {evaluation_results[0]},\nvalidation Accuracy: {evaluation_results[1]}")

# ręczne sprawdzenie modelu na danych spoza zbioru
# łatwy przykład - obraz wyglądający jak dane uczące
image_path = ".\\F1 teams\\test.jpg" # oczekiwana klasa: Real Madrid

# przekształcenie obrazka do postaci przyjmowanej przez model
img = image.load_img(image_path, target_size=(250, 250))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

prediction = model.predict(img_array)

predicted_class = np.argmax(prediction, axis=-1)[0]
print(class_to_name[predicted_class])

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9964 - loss: 0.0041
Validation loss: 0.0022790925577282906,
validation Accuracy: 0.9980079531669617
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Ferrari
